## Исследование данных об инвестиции венчурных фондов в компании-стартапы

### Данные

Самостоятельная работа основана на базе данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы

### Цель
Проанализировать данные о фондах и инвестициях, произвести выгрузки данных и ответить на поставленные вопросы с помощью SQL.



Задание 1

Посчитайте, сколько компаний закрылось.

In [ ]:
'''
SELECT COUNT(status)
FROM company
WHERE status = 'closed'
'''

Задание 2

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [ ]:
'''
SELECT funding_total
FROM company
WHERE category_code ='news' AND country_code='USA'
ORDER BY funding_total DESC;
'''

Задание 3

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
'''
SELECT SUM(price_amount) AS price_cash
FROM acquisition
WHERE term_code='cash'
      AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) IN (2011, 2012, 2013) 
'''

Задание 4

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
'''
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
'''

Задание 5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'

In [ ]:
'''
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';
'''

Задание 6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
'''
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;
'''

Задание 7

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
'''
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at 
HAVING MIN(raised_amount)!=0
   AND MIN(raised_amount)!=MAX(raised_amount);
'''

Задание 8

Создайте поле с категориями:

Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.

Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.

Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
'''
SELECT *,
     CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
        WHEN invested_companies <= 20 THEN 'low_activity'
     END
FROM fund;
'''

Задание 9

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие.

Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
'''
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds))
'''

Задание 10

Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.

Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему.

Для фильтрации диапазона по годам используйте оператор BETWEEN.

In [ ]:
'''
SELECT country_code,
       MIN(invested_companies) AS min_count,
       MAX(invested_companies) AS max_count,
       AVG(invested_companies) AS avg_count
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012 
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY avg_count DESC
LIMIT 10;
'''

Задание 11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
'''
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id;
'''

Задание 12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники.

Выведите название компании и число уникальных названий учебных заведений.

Составьте топ-5 компаний по количеству университетов.

In [ ]:
'''
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
INNER JOIN people AS p ON c.id=p.company_id
INNER JOIN education AS e ON p.id=e.person_id
GROUP BY c.name
ORDER BY COUNT(e.instituition) DESC
LIMIT 5;
'''

Задание 13

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
'''
SELECT name
FROM company
WHERE status='closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round=1 AND is_last_round=1);
'''             

Задание 14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
'''
SELECT id
FROM people
WHERE company_id IN (
                 SELECT id
                 FROM company
                 WHERE status='closed'
                 AND id IN (
                        SELECT company_id
                        FROM funding_round
                        WHERE is_first_round=1 AND is_last_round=1
                        )
                    );
'''

Задание 15

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
'''
SELECT DISTINCT p.id,
       e.instituition 
FROM people AS p
INNER JOIN education AS e 
ON p.id=e.person_id
WHERE p.company_id IN (
                   SELECT id
                   FROM company
                   WHERE status='closed'
                   AND id IN (
                          SELECT company_id
                          FROM funding_round
                          WHERE is_first_round=1 AND is_last_round=1
                              )
                      );
'''

Задание 16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания.

In [ ]:
'''
SELECT p.id,
       COUNT(e.instituition) 
FROM people AS p
INNER JOIN education AS e 
ON p.id=e.person_id
WHERE p.company_id IN (
                   SELECT id
                   FROM company
                   WHERE status='closed'
                   AND id IN (
                          SELECT company_id
                          FROM funding_round
                          WHERE is_first_round=1 AND is_last_round=1
                             )
                       )
GROUP BY p.id;
'''

Задание 17

Дополните предыдущий запрос и выведите среднее число учебных заведений, которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
'''
WITH
count_institute AS (
                    SELECT p.id,
                           COUNT(e.instituition) 
                    FROM people AS p
                    INNER JOIN education AS e
                    ON p.id=e.person_id
                    WHERE p.company_id IN (
                                       SELECT id
                                       FROM company
                                       WHERE status='closed'
                                       AND id IN (
                                              SELECT company_id
                                              FROM funding_round
                                              WHERE is_first_round=1 AND is_last_round=1
                                                 )
                                          )
                    GROUP BY p.id
                    )
 
SELECT AVG(count)
FROM (SELECT count FROM count_institute) AS avg_count;
'''

Задание 18

Напишите похожий запрос: выведите среднее число учебных заведений, которые окончили сотрудники компании Facebook.

In [ ]:
'''
WITH
count_institute AS (SELECT p.id,
                           COUNT(e.instituition) 
                    FROM people AS p
                    INNER JOIN education AS e 
                    ON p.id=e.person_id
                    WHERE p.company_id IN (
                                        SELECT id
                                        FROM company
                                        WHERE name='Facebook'
                                          )
                    GROUP BY p.id)
 
SELECT AVG(count)
FROM (SELECT count FROM count_institute) AS avg_count;
'''

Задание 19

Составьте таблицу из полей:

name_of_fund — название фонда;

name_of_company — название компании;

amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
'''
SELECT f.name as name_of_fund,
       c.name as name_of_company,
       fr.raised_amount as amount
FROM investment as i
   LEFT JOIN company as c on i.company_id = c.id
    LEFT JOIN fund as f on i.fund_id = f.id         
    LEFT JOIN funding_round as fr on fr.id = i.funding_round_id
          WHERE EXTRACT(YEAR FROM CAST(funded_at as date)) BETWEEN '2012' AND '2013'
          AND c.milestones > 6
'''

Задание 20

Выгрузите таблицу, в которой будут такие поля:

название компании-покупателя;

сумма сделки;

название компании, которую купили;

сумма инвестиций, вложенных в купленную компанию;

доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в алфавитном порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
''''
SELECT c.name as name_acquiring_company,
       a.price_amount,
       cm.name as name_acquired_company_id,
       cm.funding_total,
      round(a.price_amount/ cm.funding_total) as percent
FROM  acquisition as a
      LEFT JOIN company as c on a.acquiring_company_id = c.id
      LEFT JOIN company as cm on a.acquired_company_id = cm.id
      WHERE a.price_amount > 0
      AND cm.funding_total >0
ORDER BY  a.price_amount desc
LIMIT 10;
''''

Задание 21

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год.

Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
''''
SELECT c.name as name_company,
       EXTRACT(MONTH FROM fr.funded_at) as month_round
FROM company as c
    LEFT JOIN funding_round as fr on fr.company_id = c.id
        WHERE c.category_code = 'social'
          AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN '2010' AND '2013'
           AND fr.raised_amount != 0
''''           

Задание 22

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды.

Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

номер месяца, в котором проходили раунды;

количество уникальных названий фондов из США, которые инвестировали в этом месяце;

количество компаний, купленных за этот месяц;

общая сумма сделок по покупкам в этом месяце.

In [ ]:
''''
with
tab_1 as (
         select extract(month from fr.funded_at) as month_round,
         count(distinct f.name) as fund_name_USA
         from funding_round as fr 
         left join investment as i on fr.id = i.funding_round_id
         left join fund as f on i.fund_id = f.id
         where f.country_code ='USA'
           and extract(year from fr.funded_at) between '2010' and '2013'
         group by extract(month from fr.funded_at)
         ),
tab_2 as (
         select extract(month from a.acquired_at) as month_sale,
                count(a.acquired_company_id) as count_company,
         sum(a.price_amount) as total_price_amount
         from acquisition as a
         where extract(year from a.acquired_at) between '2010' and '2013'
         group by extract(month from a.acquired_at)
         )

select tab_1.month_round,
       tab_1.fund_name_USA,
       tab_2.count_company,
       tab_2.total_price_amount
from tab_1 left join  tab_2 on tab_1.month_round =  tab_2.month_sale;  
''''

Задание 23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах.

Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
''''
with
tab_1 as (
          select country_code ,
          avg(funding_total) as year_2011
          from company 
          where extract(year from founded_at) = '2011'
          group by country_code
          ),

tab_2 as (
          select country_code,
          avg(funding_total) as year_2012
          from company 
          where extract(year from founded_at) = '2012'
          group by country_code
          ),
tab_3 as (
          select country_code,
          avg(funding_total) as year_2013
          from company 
          where extract(year from founded_at) = '2013'
          group by country_code
          )    
          
select tab_1.country_code,
       tab_1.year_2011,
       tab_2.year_2012,
       tab_3.year_2013
from tab_1 inner join tab_2 on tab_1.country_code = tab_2.country_code
           inner join tab_3 on tab_1.country_code = tab_3.country_code
order by year_2011 desc;  
''''